In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a given youtube video.

For other deep-learning Colab notebooks, visit [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).


## Install OpenPose

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  


In [ ]:
# !rm openpose.avi
# %cd openpose
# !cd /content/openpose && ./build/examples/openpose/openpose.bin --image_dir "/content/drive/Othercomputers/My PC/cs7643-project/Images/Akarna_Dhanurasana/" --write_json "/content/drive/Othercomputers/My PC/cs7643-project/output/Akarna_Dhanurasana/" --display 0 --render_pose 0
!for d in /content/drive/Othercomputers/My\ PC/cs7643-project/Images/*; do x=$(echo "$d" | sed -E "s|(.*Images\/)(.*)|\2|g"); ./build/examples/openpose/openpose.bin --image_dir "$d" --write_json "/content/drive/Othercomputers/My PC/cs7643-project/output/$x" --display 0 --render_pose 0; done


## ETL

In [60]:
import os
import json
import pandas as pd

root = "/content/drive/Othercomputers/My PC/cs7643-project/output/"
num_points = 75
df = pd.DataFrame(columns = [f"x{i}" for i in range(num_points)]+["Y"])
pose_class_map = {}
poses_n = 0
for subdir,dir,files in os.walk(root):
    poseName = subdir.split(root,1)[1]

    for fileName in files:
        #only look for json files
        if fileName.endswith(".json"):
            p = os.path.join(root,subdir,fileName)
            with open(p,'r') as f:
                jsonData = json.load(f)
                people = jsonData["people"]
                if len(people) > 0:
                    if poseName not in pose_class_map:
                        pose_class_map[poseName] = poses_n
                        poses_n +=1
                    points = jsonData["people"][0]['pose_keypoints_2d'] + [pose_class_map[poseName]]
                    s = pd.Series(points, index = df.columns)
                    df = df.append(s, ignore_index=True)


In [62]:
df.to_csv(root+"pose_df.csv")

In [64]:
df.tail(5)


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,Y
9045,395.957,114.612,0.881803,442.897,191.312,0.074482,0.000,0.000,0.000000,0.000,0.000,0.000000,0.000,0.000,0.000000,457.806,194.529,0.295513,405.580,276.590,0.149956,0.000,0.000,0.000000,0.000,0.000,0.000000,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,404.512,99.6431,0.105450,415.218,107.098,0.855298,0.000,0.000,0.000000,465.300,139.083,0.840567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0
9046,336.460,222.130,0.720797,336.534,254.970,0.363661,337.915,257.835,0.315230,360.843,269.272,0.092640,416.609,279.285,0.137946,339.362,250.702,0.213390,0.000,0.000,0.000000,0.000,0.000,0.000000,377.966,273.600,0.055598,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,326.540,222.1050,0.799649,0.000,0.000,0.000000,316.523,233.612,0.788296,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0
9047,309.534,343.652,0.891106,278.060,448.454,0.611892,173.360,437.928,0.321988,121.007,553.187,0.597422,304.347,597.702,0.098887,364.594,443.113,0.432564,503.385,450.967,0.267134,592.397,516.518,0.422197,469.285,618.645,0.174697,490.270,613.394,0.105001,0.0,0.0,0.0,0.0,0.0,0.0,435.292,636.999,0.110670,0.0,0.0,0.0,0.0,0.0,0.0,280.709,325.3700,0.960783,314.849,327.855,0.835668,220.437,364.682,0.934186,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0
9048,243.855,222.122,0.924406,252.006,249.327,0.709555,197.663,250.695,0.626923,117.461,310.487,0.794298,200.342,344.543,0.756402,300.974,247.967,0.665454,375.820,320.061,0.827566,285.992,339.074,0.758977,256.148,412.488,0.187674,212.593,416.546,0.166149,0.0,0.0,0.0,0.0,0.0,0.0,300.978,413.836,0.165934,0.0,0.0,0.0,0.0,0.0,0.0,228.942,208.5300,0.853924,256.169,208.511,0.886553,212.588,211.304,0.888316,277.907,212.629,0.932822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0
9049,852.703,480.523,0.230800,821.393,529.472,0.126220,862.522,562.781,0.065409,0.000,0.000,0.000000,0.000,0.000,0.000000,791.952,505.942,0.193855,841.019,353.171,0.814219,925.217,490.241,0.542921,0.000,0.000,0.000000,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,846.823,480.5290,0.090487,844.887,480.547,0.214219,854.654,537.304,0.063212,844.902,535.319,0.563418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0
